<a href="https://colab.research.google.com/github/apetros/pyeplan/blob/master/examples/1_bus_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a microgrid considering all generation and load connected to single bus

This simple example neglects the network model and considers a single node with total load. One representative day with is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, wind units, solar units and diesel generation units. It showcases how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the microgrid network.

This example is designed to be run on Google Colab.

## Preparing the platform to execute the code

These commands make sure to remove any files remaining from previous executions.

In [1]:
!rm -rf sample_data
!rm -rf 1_bus

The following command installs SVN and copies the project files from Github to Colab for execution.

In [2]:
!apt-get -qq -y install subversion
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/1_bus

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: line 1: svn: command not found


The following command installs the necessary optimization solvers for pyeplan to use.

In [3]:
!apt-get -y -qq install glpk-utils coinor-cbc

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


The following command installs PyEPLAN.

In [4]:
!pip install -q pyeplan

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.21.6 which is incompatible.


This command is only used during debugging. To be removed at the final stage.

In [5]:
!pip install git+https://github.com/SPS-L/pyeplan.git

  Cloning https://github.com/SPS-L/pyeplan.git to /tmp/pip-req-build-5zj7dhb8
  Running command git clone -q https://github.com/SPS-L/pyeplan.git /tmp/pip-req-build-5zj7dhb8
  Created wheel for pyeplan: filename=pyeplan-0.4.3-py3-none-any.whl size=16405 sha256=763508b7e0ffc24fb18b372a91c79179e4d816cbffc78a068ab2313b4711bf14
  Stored in directory: /tmp/pip-ephem-wheel-cache-rdv4o474/wheels/14/b2/f5/febaab2d1c00715e387c9cd0e0ef430b7de263a913f0f88623
Successfully built pyeplan
  Attempting uninstall: pyeplan
    Found existing installation: pyeplan 0.4.2
    Uninstalling pyeplan-0.4.2:
      Successfully uninstalled pyeplan-0.4.2


## Defining the input data

The next step is to define the directory to the folder where the input data is located and define the reference node. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).

Some of the data inputs include:

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [6]:
import pandas as pd
pd.read_csv("1_bus/prep_dist.csv")

FileNotFoundError: [Errno 2] No such file or directory: '1_bus/prep_dist.csv'

### Defining the input investment candidates

#### Wind Units

In [ ]:
pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


#### Solar PV Units

In [ ]:
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


#### Diesel/Fossil Units

In [ ]:
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,0,0,0,NaN
1,0,12,0.4,0,0,0,0,0,NaN


## Case studies
Case 1: No diesel candidates

Case 2: Solar + Wind + diesel candidated

Case 3: Total load reduced from 90 kW to 70 kW

## Using the invesment and operation planning module from PyEPLAN 

In [ ]:
import pyeplan as pye

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [ ]:
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = nominal. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= nominal.

In [ ]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = True, )

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpvmngxif3.glpk.raw --wglp /tmp/tmpo21pfrkn.glpk.glp --cpxlp
 /tmp/tmppb_8nk6_.pyomo.lp
Reading problem data from '/tmp/tmppb_8nk6_.pyomo.lp'...
/tmp/tmppb_8nk6_.pyomo.lp:248: warning: lower bound of variable 'xg(0)' redefined
/tmp/tmppb_8nk6_.pyomo.lp:248: warning: upper bound of variable 'xg(0)' redefined
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
254 lines were read
Writing problem data to '/tmp/tmpo21pfrkn.glpk.glp'...
217 lines were written
GLPK Integer Optimizer, v4.65
42 rows, 31 columns, 81 non-zeros
6 integer variables, all of which are binary
Preprocessing...
9 rows, 14 columns, 28 non-zeros
6 integer variables, all of which are binary
Scaling...
 A: min|aij| =  3.333e-01  max|aij| =  4.000e+01  ratio =  1.200e+02
GM: min|aij| =  7.598e-01  max|aij| =  1.316e+00  ratio =  1.732e+00
EQ: min|aij| =  5.774e-01  max|aij| =  1.000e+00  ratio =  1.732e+00

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total Investment and Operational Costs

In [ ]:
inosys.resCost()

,total costs,20200.0
0,total investment costs,20200.0
1,total operation costs,0.0


### Number and capacity of wind units installed

The capacity and location of wind units installed is:

In [ ]:
inosys.resWind()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,0.0,0


### Number and capacity of solar units installed

The capacity and location of solar units installed is:

In [ ]:
inosys.resSolar()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,50.0,0


### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [ ]:
inosys.resConv()

,Installed Capacity (kW),Bus
Unit,,
1,0.0,0
2,0.0,0


### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [ ]:
inosys.resCurt()

,0
Hour,
0,1.184238e-16
